## connect

In [1]:
import pandas as pd
import sqlite3

/Users/a19028558/Desktop/transfer_accounts/akovyaz/learning_myself/env_school/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [5]:
connect = sqlite3.connect('../data/checking-logs.sqlite')

## create table datamart

In [6]:
pd.io.sql.read_sql("""select checker.uid, labname, datetime(timestamp) as first_commit_ts, datetime(datetime) as first_view_ts from checker
left JOIN (select * from pageviews GROUP BY uid
HAVING MIN(datetime)
ORDER BY datetime) pv on pv.uid = checker.uid
where status='ready' and numTrials=1 and labname in ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1') and checker.uid LIKE 'user_%';""", con=connect).to_sql('datamart', con=connect)

## Create pandas tables

In [7]:
test = pd.io.sql.read_sql("select uid, labname,first_commit_ts, first_view_ts from datamart where first_view_ts is not null;", con=connect, parse_dates=['first_commit_ts', 'first_view_ts'])

In [8]:
control = pd.io.sql.read_sql("select uid, labname,first_commit_ts, first_view_ts from datamart where first_view_ts is null;", con=connect,
                   parse_dates=['first_commit_ts', 'first_view_ts']).fillna(test.first_view_ts.mean())

In [10]:
test.to_sql('test', connect)
control.to_sql('control', connect)


## close connect

In [ ]:
connect.close()